In [1]:
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import sklearn.model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

# Import feature tables

In [2]:
path = os.getcwd() + '/features/'

def fileList():
    matches = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(('.csv')):
                matches.append(os.path.join(root, filename))
    return matches

list_of_files = fileList()
list_of_files

['C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/1000BoWords.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/200BoWords.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/500BoWords.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/50BoWords.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/coefs.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/coefs_May_apr.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/DynamicepertimeFeature.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/features/just_ass_rules.csv',
 'C:\\Users\\surface\\OneDrive\\DataScience_Homework\\DataMining\\project\\DataMining/feat

# Qi - Fay - Joy features

In [53]:
label = pd.read_csv(list_of_files[-1]).rename(index=str, columns={"ITEST_id": "itest_id"})
qi_f = pd.read_csv(list_of_files[9])
df = qi_f.merge(label, on="itest_id", how="inner")
df = df.drop(['MCAS','AveCorrect'],axis=1)
df = df.dropna(axis=0, how='any')

# models

In [294]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
import math
from sklearn import metrics
from sklearn.model_selection import cross_val_score


def test_LogisticRegression(X,y,K=4):
    l1_reg_c = np.arange(0.2, 0.7, 1e-2)
    score_l = []
    RMSE_l = []
    for l1 in l1_reg_c:
        model = LogisticRegression(C= l1, penalty="l1", dual=False)
        score_l.append(np.mean(cross_val_score(model, X, y,scoring='roc_auc',cv = K)))
        RMSE_l.append(-np.mean(cross_val_score(model, X, y,scoring='neg_mean_squared_error',cv = K)))
    
    score = max(score_l)
    l1 = l1_reg_c[score_l.index(score)]
#     RMSE = math.sqrt(RMSE_l[score_l.index(score)])
    RMSE = RMSE_l[score_l.index(score)]
    print("LogisticRegression l1 = %f, AUC: %f, RMSE: %f, score: %f" % (l1,score,RMSE,(1-RMSE)+score))
    
def test_SVC(X,y,K=4):
    model = SVC()
    score = np.mean(cross_val_score(model, X, y,cv = K))
    RMSE = math.sqrt(-np.mean(cross_val_score(model, X, y,scoring='neg_mean_squared_error',cv = K)))
    RMSE = -np.mean(cross_val_score(model, X, y,scoring='neg_mean_squared_error',cv = K))
    print("SVM  AUC: %f, RMSE: %f, score: %f" % (score,RMSE,(1-RMSE)+score))
    
def test_RandomForestClassifier(X,y,K=4):
    dep = range(2,20)
    dep = [11]
    score_l = []
    RMSE_l = []
    for depth in dep:
        model = RandomForestClassifier(max_depth=depth, random_state=0)
        score = np.mean(cross_val_score(model, X, y,cv = K))
        score_l.append(score)
        RMSE_l.append(-np.mean(cross_val_score(model, X, y,scoring='neg_mean_squared_error',cv = K)))
        
    
    score = max(score_l)
    RMSE = RMSE_l[score_l.index(score)]
    depth = dep[score_l.index(score)]
    print("RandomForest depth = %d, AUC: %f, RMSE: %f, score: %f" % (depth,score,RMSE,(1-RMSE)+score))
    
def test_XGB(X,y,K=4):
    dep = range(3,10)
#     dep = [11]
    score_l = []
    RMSE_l = []
    for depth in dep:
        model = xgb.XGBClassifier(max_depth=depth)
        score = np.mean(cross_val_score(model, X, y,cv = K))
        score_l.append(score)
        RMSE_l.append(-np.mean(cross_val_score(model, X, y,scoring='neg_mean_squared_error',cv = K)))
    
    score = max(score_l)
    depth = dep[score_l.index(score)]
#     RMSE = math.sqrt(RMSE_l[score_l.index(score)])
    RMSE = RMSE_l[score_l.index(score)]
    print("XGB depth = %d, AUC: %f, RMSE: %f, score: %f " % (depth,score,RMSE,(1-RMSE)+score))

# train_by_all

In [311]:
tran_frame = pd.concat([df[df['isSTEM']==0].sample(df['isSTEM'].sum()), df[df['isSTEM']==1]])
tran_frame = tran_frame.drop(['itest_id'],axis=1)
K=3
X = tran_frame.iloc[:,:-1].values
y = tran_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

LogisticRegression l1 = 0.670000, AUC: 0.540853, RMSE: 0.459469, score: 1.081384
SVM  AUC: 0.500000, RMSE: 0.500000, score: 1.000000
RandomForest depth = 11, AUC: 0.549790, RMSE: 0.450210, score: 1.099581


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 5, AUC: 0.552935, RMSE: 0.447065, score: 1.105870 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [261]:
# reduction
from sklearn.decomposition import PCA
# dimensionality reduction
def dimension_reduction(data,K):
    pca = PCA(n_components=K)
    return pca.fit_transform(data)

In [315]:
K=3
X = tran_frame.iloc[:,:-1].values
X = dimension_reduction(X,45)
y = tran_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

LogisticRegression l1 = 0.690000, AUC: 0.544089, RMSE: 0.475250, score: 1.068839
SVM  AUC: 0.500000, RMSE: 0.500000, score: 1.000000
RandomForest depth = 11, AUC: 0.534242, RMSE: 0.465758, score: 1.068484


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 5, AUC: 0.531039, RMSE: 0.468961, score: 1.062078 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# train after selection

In [316]:
basic = ["aveknow","avecarelessness","avecorrect","numactions","averesbored",
         "averesengcon","averesconf","averesfrust","averesofftask","averesgaming"]
K=5
X = tran_frame.iloc[:,:-1][basic].values
y = tran_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

LogisticRegression l1 = 0.420000, AUC: 0.546484, RMSE: 0.475000, score: 1.071484
SVM  AUC: 0.518750, RMSE: 0.481250, score: 1.037500
RandomForest depth = 11, AUC: 0.478125, RMSE: 0.521875, score: 0.956250


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 4, AUC: 0.534375, RMSE: 0.465625, score: 1.068750 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [121]:
from minepy import MINE
#由于MINE的设计不是函数式的，定义mic方法将其为函数式的，
# 返回一个二元组，二元组的第2项设置成固定的P值0.5
def mic(x, y):
    m = MINE()
    m.compute_score(x, y)
    return m.mic()

def information_selector(dataframe):
    v = dataframe.values
    MIC = []
    for i in range(107):
        MIC.append(mic(v[:,i],v[:,-1]))
    x = np.array(list(dataframe.columns))[:-1]
#     print(x)
    MIC_df = pd.DataFrame(np.asarray(MIC),index=x,columns=['MIC'])
    return MIC_df

In [317]:
MIC_importance = information_selector(tran_frame)
# MIC_importance.sort_values(by='MIC', ascending=False)
MIC_2_feature_name = MIC_importance[MIC_importance['MIC']>0.25].index.tolist()

In [318]:
# train
print(MIC_2_feature_name)
K=3
X = tran_frame.iloc[:,:-1][MIC_2_feature_name].values
X = scaler_std(X)
y = tran_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

['averesofftask', 'actioncount', 'hintrate', 'timetohint', 'timetohint_M', 'timetaken_H', 'std_questions_by_mon', 'std_questions_by_day_M']
LogisticRegression l1 = 0.200000, AUC: 0.602588, RMSE: 0.428022, score: 1.174565
SVM  AUC: 0.568658, RMSE: 0.431342, score: 1.137317
RandomForest depth = 11, AUC: 0.562194, RMSE: 0.437806, score: 1.124389


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 3, AUC: 0.590613, RMSE: 0.409387, score: 1.181225 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# extent features

In [160]:
# scaler
from sklearn.preprocessing import StandardScaler
def scaler_std(data):
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

In [319]:
from sklearn.preprocessing import PolynomialFeatures
X = tran_frame.iloc[:,:-1][MIC_2_feature_name].values
poly = PolynomialFeatures(2)
new_X = poly.fit_transform(X)
new_columns = poly.get_feature_names(MIC_2_feature_name)
new_frame = pd.DataFrame(data = new_X,index = tran_frame.index,columns = new_columns)
new_frame['isSTEM'] = tran_frame.iloc[:,-1].values
new_frame

,1,averesofftask,actioncount,hintrate,timetohint,timetohint_M,timetaken_H,std_questions_by_mon,std_questions_by_day_M,averesofftask^2,averesofftask actioncount,averesofftask hintrate,averesofftask timetohint,averesofftask timetohint_M,averesofftask timetaken_H,averesofftask std_questions_by_mon,averesofftask std_questions_by_day_M,actioncount^2,actioncount hintrate,actioncount timetohint,actioncount timetohint_M,actioncount timetaken_H,actioncount std_questions_by_mon,actioncount std_questions_by_day_M,hintrate^2,hintrate timetohint,hintrate timetohint_M,hintrate timetaken_H,hintrate std_questions_by_mon,hintrate std_questions_by_day_M,timetohint^2,timetohint timetohint_M,timetohint timetaken_H,timetohint std_questions_by_mon,timetohint std_questions_by_day_M,timetohint_M^2,timetohint_M timetaken_H,timetohint_M std_questions_by_mon,timetohint_M std_questions_by_day_M,timetaken_H^2,timetaken_H std_questions_by_mon,timetaken_H std_questions_by_day_M,std_questions_by_mon^2,std_questions_by_mon std_questions_by_day_M,std_questions_by_day_M^2,isSTEM
13,1.0,0.185330,1.976027,0.178739,12.417808,11.804196,85.292308,21.815242,5.055054,0.034347,0.366218,0.033126,2.301395,2.187674,15.807239,4.043023,0.936854,3.904684,0.353192,24.537929,23.325414,168.539937,43.107515,9.988925,0.031947,2.219541,2.109865,15.245026,3.899225,0.903533,154.201961,146.582239,1059.143519,270.897485,62.772692,139.339038,1006.807100,257.511382,59.670848,7274.777759,1860.672292,431.157225,475.904762,110.277225,25.553571,0
144,1.0,0.182188,2.520000,0.171322,14.395556,14.768707,63.000000,32.101402,13.265826,0.033192,0.459113,0.031213,2.622690,2.690674,11.477813,5.848474,2.416868,6.350400,0.431731,36.276800,37.217143,158.760000,80.895533,33.429882,0.029351,2.466275,2.530204,10.793283,5.499675,2.272727,207.232020,212.603749,906.920001,462.117514,190.968937,218.114721,930.428572,474.096213,195.919106,3969.000008,2022.388318,835.747047,1030.500000,425.851615,175.982143,0
466,1.0,0.197900,1.678161,0.040501,4.659004,2.338983,59.052632,47.744982,8.952254,0.039164,0.332108,0.008015,0.922017,0.462885,11.686524,9.448738,1.771652,2.816224,0.067967,7.818558,3.925190,99.099818,80.123763,15.023323,0.001640,0.188694,0.094731,2.391687,1.933716,0.362575,21.706317,10.897331,275.126437,222.444055,41.708587,5.470842,138.123105,111.674705,20.939171,3487.213291,2819.466847,528.654175,2279.583333,427.425223,80.142857,0
125,1.0,0.295400,3.084507,0.139763,17.957746,14.880952,139.571428,15.631165,6.928203,0.087261,0.911163,0.041286,5.304717,4.395832,41.229388,4.617445,2.046591,9.514184,0.431101,55.390795,45.900402,430.509054,48.214440,21.370092,0.019534,2.509834,2.079811,19.506967,2.184664,0.968309,322.480659,267.228370,2506.388328,280.700506,124.414917,221.442744,2076.955780,232.606629,103.098262,19480.183646,2181.664091,966.979222,244.333333,108.295891,48.000000,0
427,1.0,0.179375,1.578652,0.032437,2.134831,2.769663,67.870968,52.647977,12.472534,0.032176,0.283171,0.005818,0.382936,0.496809,12.174378,9.443749,2.237265,2.492141,0.051207,3.370155,4.372333,107.144618,83.112818,19.689787,0.001052,0.069248,0.089840,2.201540,1.707749,0.404573,4.557505,5.912764,144.893077,112.394559,26.626758,7.671033,187.979703,145.817151,34.544715,4606.468260,3573.269175,846.522948,2771.809524,656.653684,155.564103,0
27,1.0,0.269405,1.850000,0.042235,6.875000,8.984127,70.851852,20.330601,6.913754,0.072579,0.498400,0.011378,1.852161,2.420371,19.087863,5.477171,1.862602,3.422500,0.078135,12.718750,16.620635,131.075926,37.611612,12.790446,0.001784,0.290368,0.379448,2.992452,0.858670,0.292005,47.265625,61.765873,487.106481,139.772881,47.532062,80.714538,636.542034,182.652700,62.114048,5019.984909,1440.460723,489.852303,413.333333,140.560782,47.800000,0
398,1.0,0.123936,2.069395,0.097186,5.074733,3.162712,47.679389,74.233223,16.824509,0.015360,0.256473,0.012045,0.628943,0.391974,5.909197,9.200176,2.085164,4.282396,0.201115,10.501627,6.544900,98.667491,153.617862,34.816555,0.009445,0.493191,0.307370,4.633749,7.214398,1.

In [320]:
# train
K=3
X = new_frame.iloc[:,:-1].values
X = scaler_std(X)
X = dimension_reduction(X,20)
y = new_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

LogisticRegression l1 = 0.690000, AUC: 0.578879, RMSE: 0.468845, score: 1.110035
SVM  AUC: 0.587701, RMSE: 0.412299, score: 1.175402
RandomForest depth = 11, AUC: 0.478046, RMSE: 0.521954, score: 0.956091


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 5, AUC: 0.525215, RMSE: 0.474785, score: 1.050431 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [321]:
# basic extent

X = tran_frame.iloc[:,:-1][basic].values
poly = PolynomialFeatures(2)
new_X = poly.fit_transform(X)
new_columns = poly.get_feature_names(basic)
new_frame = pd.DataFrame(data = new_X,index = tran_frame.index,columns = new_columns)
new_frame['isSTEM'] = tran_frame.iloc[:,-1].values


In [322]:
K=3
X = new_frame.iloc[:,:-1].values
X = scaler_std(X)
X = dimension_reduction(X,20)
y = new_frame.iloc[:,-1].values
test_LogisticRegression(X,y,K)
test_SVC(X,y,K)
test_RandomForestClassifier(X,y,K)
test_XGB(X,y,K)

LogisticRegression l1 = 0.220000, AUC: 0.552976, RMSE: 0.471756, score: 1.081220
SVM  AUC: 0.472513, RMSE: 0.527487, score: 0.945027
RandomForest depth = 11, AUC: 0.475076, RMSE: 0.524924, score: 0.950151


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning

XGB depth = 3, AUC: 0.515840, RMSE: 0.484160, score: 1.031679 


C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\surface\Anaconda2\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
